## Scraping Google Reviews using Selenium

### Used selenium to scrape the data as amazon now blocks bots, scrappers and crawlers. 
### Amazon blocks it and the scrappers throws an error that indicates amazon blocks it.  

In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import time
import re

options = Options()
DRIVER_PATH = '/path/to/chromedriver'
options.headless = True
options.add_argument("--window-size=1920,1200")
driver = webdriver.Chrome(options=options, executable_path=DRIVER_PATH)

#Function to set the target column
def sentiment(rate):
    rate = int(rate[0])
    if rate > 3 : 
        label='Good'
    elif rate ==3:
        label='Normal'
    else:
        label='Bad'
    return label

#Getting the number of people that found the coment helpful
def helfulness(x):
    try:
        element = x.find_element(By.XPATH,".//span[@data-hook='helpful-vote-statement']")
        txt = element.get_attribute('innerHTML')
        helpful = re.search(r'(.*?)person found this helpful', txt).group(1).strip()
    except:
        try:
            helpful = re.search(r'(.*?)people found this helpful', txt).group(1).strip()
            print(helpful)
        except:
            helpful = '0'
    return helpful
  

#Setting the columns for Dataframe
cols = ['comments', 'rating', 'helpful', 'label']
amazon = pd.DataFrame(columns=cols)


#k is page number that loops through all the pages of comments
k=1 

#If there is any error break the process
brk=False

while True:
    #Go to the amazon site
    driver.get('https://www.amazon.ca/Rockstar-Games-E3GrandTheftAutoVplaystation_4-Grand-Theft/product-reviews/B00KVSQ848/ref=cm_cr_getr_d_paging_btm_prev_1?ie=UTF8&reviewerType=all_reviews&pageNumber={a}'.format(a=k))
    #Wait ome time to load the page
    time.sleep(5)
    
    
    try:
        #Wait until reviews are visible 
        reviews=WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "//div[@id='cm_cr-review_list']")))
    except:
        #save a screenshot to see if there's any error
        driver.save_screenshot('screenshot{j}.png'.format(j=k))
        break
    
    
    #Each page has 10 reviews
    for i in range(1,11):
        
        try:
            xpath = '(//div[@data-hook="review"])[{j}]'.format(j=i)
        except:
            brk=True
            break
            
        try:
            element = reviews.find_element(By.XPATH,xpath)
            
            #text is in the review-body span
            txt_body = element.find_element(By.XPATH, "(//div[@data-hook='review'])[{j}]//span[@data-hook='review-body']//span".format(j=i))
            txt = txt_body.get_attribute('innerHTML')
            
            #Get the helpful Number
            helpful = helfulness(element)
            
            #Get the star rating
            rate = element.find_element(By.XPATH,xpath+'//span[@class="a-icon-alt"]')#//a[@class="a-link-normal"]
            
            #get the target label
            label = sentiment(rate.get_attribute('innerHTML'))
            rate = rate.get_attribute('innerHTML')[:1]
            
        except:
            rate=0
            label = 'NA'
            helpful= '0'
            
        temp = pd.DataFrame([[txt,rate,helpful,label]], columns=cols)
        amazon = pd.concat([amazon, temp], axis=0, ignore_index=True)
        
    if brk==True:
        break
    k+=1
    

driver.quit()
#Ignore the output 

C:\Users\nikhi\AppData\Local\Temp\ipykernel_26216\629736000.py:35: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  options.headless = True
C:\Users\nikhi\AppData\Local\Temp\ipykernel_26216\629736000.py:37: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(options=options, executable_path=DRIVER_PATH)


3
20
4
4
4
3
2
2
5
51
148
2
5
2
7
23
4
2
4
11
2
2
8
2
2
13
5
4
2
2
4
3
2
2
2
15
3
10
2
3
4
2
4
2
2
2
2
2
23
6
14
2
5
2
4
8
4
18
2
2
4
16
2
3
2
3
2
2
2
2
6
4
4
2
3
2
2
3
20
22
3
2
2
7
2
2
2
2
6
8
5
6
15
4
3
3
2
2
13
2
5
8
12
9
2
2
13
3
21
10
12
6
2
3
2
5
8
4
3
11
5
2
2
17
2
7
33
2
3
8
8
2
37
14
4
2
14
6
8
6
8
14
12
5
3
2
22
7
23
2
5
2
7
2
2
47
2
2
13
9
2
17
3
3
3
6
9
10
2
2
2
2
5
2
3
3
3
4
5
8
5
3
8
2
6
2
3
4
3
2
2
4
7
2
8
2
2
5
2
2
3
6
5
2
4
2
2
3
2
3
2
3
2
3
3
2


In [40]:
amazon

,comments,rating,helpful,label
0,So fun and the characters are hilarious,5,0,Good
1,Arrived quickly in great condition,5,0,Good
2,Ha ha Stress relief by blowing stuff up in thi...,5,0,Good
3,Super produit livraison avant temps conforme a...,5,0,Good
4,I don't understand why this game is still play...,3,One,Normal
...,...,...,...,...
3385,Classic game love it.,5,0,Good
3386,Llegó el día especificado,5,0,Good
3387,Llegó el día especificado,0,0,NA
3388,Llegó el día especificado,0,0,NA


In [4]:
#Saving it in csv format
amazon.loc[:, "comments"] = amazon["comments"].apply(lambda x : x.replace('\n', '\\n'))
amazon.to_csv('ama_rating6.csv', index=False )

## Scraping Google Reviews using selenium

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import time

options = Options()
DRIVER_PATH = '/path/to/chromedriver'
options.headless = True
options.add_argument("--window-size=1920,1200")

driver = webdriver.Chrome(options=options, executable_path=DRIVER_PATH)
driver.get("https://www.google.com/search?q=GTA+5+reviews&sxsrf=AJOqlzWg8cBgjwNk_fvIZtizPdbtEfch0Q%3A1676996048074&ei=0O30Y7KaBPCkptQP1LyjiAE&ved=0ahUKEwiywvCNgaf9AhVwkokEHVTeCBEQ4dUDCA8&uact=5&oq=GTA+5+reviews&gs_lcp=Cgxnd3Mtd2l6LXNlcnAQAzIECCMQJzIFCAAQkQIyCQgAEAcQHhDxBDIJCAAQBxAeEPEEMgYIABAHEB4yCQgAEAcQHhDxBDIJCAAQBxAeEPEEMgkIABAHEB4Q8QQyCQgAEAcQHhDxBDIGCAAQBxAeOgoIABBHENYEELADOgcIABCwAxBDOg0IABDkAhDWBBCwAxgBOgwILhDIAxCwAxBDGAI6DwguENQCEMgDELADEEMYAjoHCAAQQxCRAkoECEEYAFDJCFiHFWCaGmgCcAF4AIABUIgB9gKSAQE1mAEAoAEByAETwAEB2gEGCAEQARgJ2gEGCAIQARgI&sclient=gws-wiz-serp")

#Wait for some time to load the page
time.sleep(2)

#Wait until the Google reviews are visible and clickable 
rev=WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "//div[contains(@class,'MXl0lf tKtwEb wHYlTd')]")))

#printing the element content
print(rev)
rev.click()
time.sleep(10)

#Saving Screenshot to 
driver.save_screenshot('screenshot.png')

#After cliking on the reviews dialog box is opened and print te reviews
reviews = driver.find_element(By.XPATH,"//g-dialog-content")
print(reviews.text)


#It only displays few reviews as it is not scrooled to bottom of dialog box.

C:\Users\nikhi\AppData\Local\Temp\ipykernel_9192\1348997723.py:9: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  options.headless = True
C:\Users\nikhi\AppData\Local\Temp\ipykernel_9192\1348997723.py:12: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(options=options, executable_path=DRIVER_PATH)


<selenium.webdriver.remote.webelement.WebElement (session="9ffe1daf0b5b649fcffd24946d249bd7", element="21f2506d-12be-4b51-b82c-7d669848f085")>
Grand Theft Auto V
Write a review
Filter by
All
Jason White
a year ago
Even 8 years later, this game is easily one of the best that you can buy.

Whilst GTA Online has gone in a direction I don't like - and wouldn't really recommend in engaging very much with to complete newcomers by this stage - I can't ignore that it still has thousands of hours to offer players across a variety of different activities - there's something for everyone in GTA Online and it will also make them money whilst they're at it.

Single Player is also undeniably a brilliant experience. Eve ...MORE
262 people found this helpful.
Helpful
Unhelpful
Connor Pearson
a year ago
Even 8 years later, this game is easily one of the best that you can buy.

Whilst GTA Online has gone in a direction I don't like - and wouldn't really recommend in engaging very much with to complete n

## Fetching data and assigning it to DataFrame

In [2]:
import pandas as pd
import re

#Function to set the target column
def sentiment(rate):
        if rate > 3 : 
            label='Good'
        elif rate ==3:
            label='Normal'
        else:
            label='Bad'
        return label

#Getting the number of people that found the coment helpful
def helfulness(x):
        if re.search(r'\n(.*?)people found this helpful', x.text) is None:
            helpful='0'
        else:
            helpful = re.search(r'\n(.*?)people found this helpful', x.text).group(1).strip()
        return helpful


    
    
#Setting the columns for Dataframe
cols = ['comments','rating','helpful','label']
GTA_five = pd.DataFrame(columns=cols)



#----------------------------------------
# Scroll down to bottom of page

# When the dialog box open it opens 24 records first and after scrolling it loads according to te scroll.
# The review elements has attrubute data-rv and its value ranges from 0-24 and reset to 0 for 25th review and so on.
i=2
last_element = reviews.find_element(By.XPATH,'//div[@data-rv="24"][1]')
driver.execute_script("arguments[0].scrollIntoView(true);", last_element)
driver.save_screenshot('screenshot0.png')
time.sleep(5)


while True:
    try:
        last_element = reviews.find_element(By.XPATH,'(//div[@data-rv="24"])[{j}]'.format(j=i))
        driver.execute_script("arguments[0].scrollIntoView(true);", last_element)
    except:
        print("fuck")
        break
    
    i+=1
    print(i)
    driver.save_screenshot('screenshot{j}.png'.format(j=i))
    time.sleep(5)

#------------------------------------------    
    

    
# Fetching the comment element and process it to store in data frame 

# The review elements has attrubute data-rv and its value ranges from 0-24 and reset to 0 for 25th review and so on.
#y is the iterator that selects the comments like for e.g y=0 means it will select first batch of 24 reviews
y=0

#Total reviews 4291
for i in range(0, 4291):
    # The review elements has attrubute data-rv and its value ranges from 0-24 and reset to 0 for 25th review and so on.
    if i%25==0:
        y+=1
    i = i%25
    
    try:
        x = reviews.find_element(By.XPATH,'(//div[@data-rv="{j}"])[{k}]'.format(j=i,k=y))
    except:
        break
    
    #Some comments have READ MORE link to expand the text. Below code handles it. 
    try:
      S  x.find_element(By.TAG_NAME,"a")
        print("found more")
        x.click()
        time.sleep(5)
        #Get the star rating
        rate = x.find_element(By.TAG_NAME,"star-rating").get_attribute("data-ir")
        #Get the label based on rating
        label = sentiment(int(rate))
        #Get the helpul number
        helpful = helfulness(x)
        
    except:
        rate = x.find_element(By.TAG_NAME,"star-rating").get_attribute("data-ir")
        label = sentiment(int(rate))
        helpful = helfulness(x)
        print("no element")
        
    print(rate,helpful) 
    
    x = reviews.find_element(By.XPATH,'(//div[@data-rv="{j}"])[{k}]'.format(j=i,k=y))
    #Creating Dataframe for comment
    temp = pd.DataFrame([[x.text,rate,helpful,label]], columns=cols)
    
    #Concat it with main df 
    GTA_five = pd.concat([GTA_five, temp], axis=0, ignore_index=True)
    
    
driver.quit()

#Ignore the output

3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
fuck
found more
5 262
found more
5 217
found more
5 51
found more
5 653
found more
5 5
found more
0 1025
found more
5 273
found more
2 87
found more
0 332
found more
5 46
found more
0 496
found more
5 16
found more
5 12
found more
5 17
found more
0 153
found more
5 0
found more
5 20
found more
5 0
found more
5 4
found more
5 46
found more
5 41
found more
5 34
found more
5 40
found more
5 25
found more
5 2
found more
5 60
found more
5 47
found more
1 111
found more
4 31
found more
5 28
found more
5 5
found more
5 0
found more
4 13
found more
5 9
found more
5 6
found more
0 51
found more
5 3
found more
5 7
found more
2 7
no element
5 0
found more
2 0
found more
5 22
found more
2 0
found more
5 2
found more
4 18
found more
5 12
found more
5 0
found more
0 100
found more
5 13
found more
4 6
no element
5 2
found

no element
5 0
no element
5 0
no element
1 2
no element
5 0
no element
3 0
no element
1 2
no element
5 0
no element
0 3
no element
1 0
no element
5 0
no element
5 0
no element
5 0
no element
5 0
no element
2 0
no element
1 0
no element
5 0
no element
1 0
no element
5 0
no element
1 0
no element
5 0
no element
5 0
no element
1 0
no element
5 0
no element
2 0
no element
5 0
no element
4 0
no element
5 0
no element
4 0
no element
5 0
no element
5 0
no element
4 0
found more
5 0
no element
5 0
no element
3 0
no element
1 0
no element
5 0
no element
5 0
no element
5 0
no element
1 2
no element
2 0
no element
1 3
no element
1 3
no element
2 0
no element
0 4
no element
5 0
no element
5 0
no element
5 0
no element
5 0
no element
5 0
no element
1 6
no element
1 0
no element
5 0
no element
1 0
no element
5 0
no element
0 0
no element
5 0
no element
0 0
no element
1 0
no element
5 0
no element
0 0
no element
0 2
no element
1 0
no element
1 2
no element
5 0
no element
5 0
no element
5 0
no element

no element
5 0
no element
5 0
no element
5 0
no element
5 0
no element
0 0
no element
5 0
no element
5 0
no element
0 0
no element
5 0
no element
4 0
no element
1 0
no element
1 0
no element
5 0
found more
5 0
no element
2 0
no element
5 0
no element
0 0
no element
5 0
no element
5 0
no element
5 0
no element
5 0
no element
1 0
no element
5 0
no element
5 0
no element
0 0
no element
0 0
no element
5 0
no element
5 0
no element
5 0
no element
0 0
no element
0 0
no element
2 0
no element
0 0
no element
5 0
no element
5 0
no element
0 2
no element
0 0
no element
5 0
no element
0 0
no element
5 0
no element
5 0
no element
5 0
no element
5 0
no element
5 0
no element
5 0
no element
0 0
no element
1 0
no element
5 0
no element
5 0
no element
5 0
no element
5 0
no element
5 0
no element
5 0
no element
0 2
no element
0 0
no element
5 0
no element
5 0
no element
0 0
no element
5 0
no element
5 0
no element
0 0
no element
5 0
no element
5 0
no element
0 0
no element
5 0
no element
5 0
no element

In [3]:
#Storing fetched data in csv
GTA_five.loc[:, "comments"] = GTA_five["comments"].apply(lambda x : x.replace('\n', '\\n'))
GTA_five.to_csv('Gta_rating.csv', index=False )